In [1]:
import os
import os.path as osp
import pandas as pd

from ddf_utils.str import to_concept_id, format_float_digits
from ddf_utils.factory import ihme

/Users/semio/.pyenv/versions/3.6.2/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
from functools import partial

In [3]:
import dask.dataframe as dd

In [4]:
from zipfile import ZipFile

# TODOs

- fix float format

In [5]:
# metadata
md = ihme.load_metadata()

In [6]:
md.keys()

dict_keys(['age', 'cause', 'groups', 'location', 'measure', 'metric', 'rei', 'sequela', 'sex', 'year', 'year_range', 'version'])

In [7]:
md['measure']

,measure_id,sort,name,short_name
1,1,1,Deaths,Deaths
2,2,4,DALYs (Disability-Adjusted Life Years),DALYs
25,25,26,Maternal mortality ratio,MMR
26,26,28,Life expectancy,LE
27,27,27,Probability of death,PoD
28,28,29,HALE (Healthy life expectancy),HALE
29,29,21,Summary exposure value,SEV
3,3,3,YLDs (Years Lived with Disability),YLDs
4,4,2,YLLs (Years of Life Lost),YLLs
43,43,43,HAQ Index (Healthcare Access and Quality Index),haq_index


In [30]:
md['location'].head()

,location_id,type,name,medium_name,short_name,enabled
1,1,NaN,Global,Global,Earth,NaN
10,10,NaN,Cambodia,Cambodia,Cambodia,NaN
100,100,NaN,High-income North America,High-income North America,HI N Am,NaN
101,101,NaN,Canada,Canada,Canada,NaN
102,102,NaN,United States,United States,USA,NaN


In [6]:
md['measure']

,measure_id,sort,name,short_name
1,1,1,Deaths,Deaths
2,2,4,DALYs (Disability-Adjusted Life Years),DALYs
25,25,26,Maternal mortality ratio,MMR
26,26,28,Life expectancy,LE
27,27,27,Probability of death,PoD
28,28,29,HALE (Healthy life expectancy),HALE
29,29,21,Summary exposure value,SEV
3,3,3,YLDs (Years Lived with Disability),YLDs
4,4,2,YLLs (Years of Life Lost),YLLs
43,43,43,HAQ Index (Healthcare Access and Quality Index),haq_index


In [50]:
md['metric'].head()

,metric_id,name,short_name
1,1,Number,#
2,2,Percent,%
3,3,Rate,Rate
5,5,Years,Years
8,8,Probability of death,Probability of death


In [16]:
md['age'].shape

(46, 6)

In [20]:
md['age'].query("type == 'specific'").shape

(25, 6)

In [74]:
ages = md['age'].query("type == 'specific'").age_id.values.tolist()

In [75]:
ages.append(22)  # All Age
ages.append(27)  # Age-standardized

In [68]:
len(ages)

26

In [76]:
for i in ages:
    print(i, end=', ')

10, 11, 12, 13, 14, 15, 155, 16, 162, 17, 18, 19, 2, 20, 21, 3, 30, 31, 32, 4, 5, 6, 7, 8, 9, 22, 27, 

In [29]:
source_dir = '../source/cause/mmr_cause_ageall/'

In [30]:
!ls $source_dir | head

IHME-GBD_2016_DATA-39c08057-1.zip


In [7]:
n = "IHME-GBD_2016_DATA-0f5cd7a0-1.zip"
f = osp.join(source_dir, n)

In [8]:
zf = ZipFile(f)

In [9]:
fname = osp.splitext(n)[0]

In [10]:
fname + '.csv'

'IHME-GBD_2016_DATA-0f5cd7a0-1.csv'

In [11]:
data = pd.read_csv(zf.open(fname + '.csv'))

In [12]:
data.sample(10)

,measure,location,sex,age,cause,metric,year,val,upper,lower
970795,1,101,2,22,491,1,1991,36428.509094,37814.720550,35209.925507
394360,1,129,1,30,468,1,1991,0.026046,0.045314,0.014452
569090,1,170,3,235,618,1,1990,0.090584,0.134470,0.063628
54516,1,190,2,23,596,1,1991,0.057264,0.200503,0.010106
2383,1,121,2,19,429,1,1992,33.613825,56.491770,15.765678
825680,1,122,1,158,503,1,1992,14.036031,18.701929,8.413387
748597,1,182,1,13,656,1,1992,0.133535,0.369846,0.029893
316456,1,153,2,13,650,1,1992,0.046454,0.081040,0.020653
313461,1,351,3,31,511,1,1990,0.001115,0.002366,0.000424
30486,1,30,1,1,688,1,1992,0.682537,1.395648,0.292053


In [13]:
data.shape

(1000000, 10)

In [22]:
data.memory_usage().sum() / 1024 / 1024

76.29402160644531

In [25]:
76 * 32

2432

In [31]:
all_data = []
for n in os.listdir(source_dir):
    print(n)
    if not n.endswith('.zip'):
        continue
    f = osp.join(source_dir, n)
    zf = ZipFile(f)
    fname = osp.splitext(n)[0] + '.csv'
    data = pd.read_csv(zf.open(fname))
    data = data.drop(['upper', 'lower'], axis=1)
    assert set(data['metric'].unique().tolist()) == set([3])
    assert data['measure'].unique().tolist() == [25]
    
    # when metric / measure have only one value, enable this line to decrease memory usage
    # data = data.drop(['metric', 'measure'], axis=1)
    all_data.append(data)
    zf.close()

IHME-GBD_2016_DATA-39c08057-1.zip


In [32]:
data_full = all_data.pop()

In [33]:
l = len(all_data)
for _ in range(l):
    data_full = data_full.append(all_data.pop(), ignore_index=True)
    # print(data_full.shape)

In [66]:
# data_full = data_full.drop_duplicates(subset=['location', 'sex', 'cause', 'age', 'year', 'measure', 'metric'])

In [34]:
data_full.shape

(576180, 8)

In [14]:
data_full.sample(10)

,location,sex,age,cause,year,val
36367360,190,1,22,642,1999,0.002420
3444513,211,2,13,718,1993,0.004536
26960771,191,3,10,387,2001,0.003558
45285128,14,2,22,353,1998,0.000000
52927178,45,3,4,716,1993,0.005300
18134674,435,1,13,294,2000,1.000000
47987758,218,2,14,847,1996,0.001109
44729740,210,2,17,523,1997,0.008213
13607689,20,3,7,713,2006,0.008149
7139209,39,2,7,295,2007,0.391554


In [38]:
data_full.age.unique()

array([27, 22])

In [39]:
data_full.year.unique()

array([2004, 2001, 2003, 2005, 2000, 2002, 1999, 2006, 2007, 1997, 1998,
       1996, 1995, 1993, 1994, 1990, 1992, 1991, 2014, 2016, 2015, 2013,
       2012, 2011, 2010, 2009, 2008])

In [40]:
data_full.metric.unique()

array([3, 1, 2])

In [41]:
data_full.measure.unique()

array([4])

In [42]:
data_full.cause.unique()

array([586, 587, 545, 546, 554, 557, 558, 560, 294, 295, 296, 297, 298,
       300, 301, 302, 318, 319, 320, 321, 322, 328, 329, 332, 333, 334,
       335, 336, 337, 338, 339, 456, 459, 462, 465, 468, 471, 474, 477,
       480, 483, 484, 485, 486, 725, 726, 727, 728, 729, 741, 842, 843,
       845, 846, 847, 848, 849, 854, 934, 935, 938, 942, 943, 944, 945,
       946, 947, 948, 949, 950, 526, 527, 528, 529, 530, 531, 532, 533,
       534, 535, 541, 542, 543, 544, 561, 562, 563, 564, 566, 507, 508,
       509, 510, 511, 512, 513, 514, 515, 516, 520, 521, 522, 523, 524,
       525, 590, 591, 592, 593, 594, 595, 596, 602, 603, 604, 605, 607,
       608, 612, 613, 614, 615, 616, 618, 619, 626, 627, 639, 640, 641,
       642, 643, 644, 645, 358, 359, 360, 361, 695, 696, 697, 698, 699,
       700, 704, 705, 706, 707, 708, 709, 710, 711, 712, 713, 715, 716,
       717, 718, 721, 723, 724, 648, 649, 650, 651, 652, 653, 656, 657,
       387, 388, 390, 391, 392, 393, 394, 395, 396, 399, 400, 40

In [35]:
groups = data_full.groupby(by=['measure', 'metric'])

In [6]:
metric = md['metric'].copy()
measure = md['measure'].copy()

In [7]:
metric = metric.set_index('metric_id')['name'].to_dict()
measure = measure.set_index('measure_id')['short_name'].to_dict()

In [27]:
measure

{1: 'Deaths',
 2: 'DALYs',
 25: 'MMR',
 26: 'LE',
 27: 'PoD',
 28: 'HALE',
 29: 'SEV',
 3: 'YLDs',
 4: 'YLLs',
 43: 'haq_index',
 5: 'Prevalence',
 6: 'Incidence'}

In [18]:
formatter = partial(format_float_digits, digits=2)

In [36]:
all_measures = []

for g in groups.groups:
    name  = measure[g[0]] + ' ' + metric[g[1]]
    # print(name)
    concept = to_concept_id(name)
    all_measures.append([concept, name])
    
    df = groups.get_group(g)
    df = df.rename(columns={'val': concept})
    cause_groups = df.groupby(by='cause')  # split by cause
    cols = ['location', 'sex', 'age', 'cause', 'year', concept]
    df[concept] = df[concept].map(formatter)
    # if concept == 'mmr_rate':
    #     print(df.sex.unique())
    for g_ in cause_groups.groups:
        df_ = cause_groups.get_group(g_)
#         print(g_)
#         print(df_.age.unique())
#         print(len(df_.year.unique()))
#         print(len(df_.location.unique()))
        cause = 'cause-{}'.format(g_)
        by = ['location', 'sex', 'age', cause, 'year']
        file_name = 'ddf--datapoints--'+concept+'--by--'+'--'.join(by)+'.csv'
        file_path = osp.join('../../../ddf--ihme--global_burden_disease_maternal_mortality/', file_name)
        df_[cols].sort_values(by=['location', 'sex', 'age', 'year']).to_csv(file_path, index=False)

In [76]:
measure[1] + ' ' + metric[3]

'Deaths Rate'

In [45]:
# data_full = data_full.drop(['measure', 'metric'], axis=1)

In [46]:
# data_full['val'] = data_full['val'].map(format_float_digits)

In [37]:
all_measures = []

In [77]:
name  = measure[1] + ' ' + metric[3]
# print(name)
concept = to_concept_id(name)
# all_measures.append([concept, name])

cause_groups = data_full.groupby(by='cause')  # split by cause
cols = ['location', 'sex', 'age', 'cause', 'year', concept]

In [78]:
# if concept == 'mmr_rate':
#     print(df.sex.unique())
for g_ in cause_groups.groups:
    df_ = cause_groups.get_group(g_)
    df_  = df_.rename(columns={'val': concept})
    # df_ = df_[df_.age.isin(ages)]
    df_[concept] = df_[concept].map(formatter)
#         print(g_)
#         print(df_.age.unique())
#         print(len(df_.year.unique()))
#         print(len(df_.location.unique()))
    cause = 'cause-{}'.format(g_)
    by = ['location', 'sex', 'age', cause, 'year']
    file_name = 'ddf--datapoints--'+concept+'--by--'+'--'.join(by)+'.csv'
    file_path = osp.join('../../cause/deaths/', file_name)
    df_[cols].sort_values(by=['location', 'sex', 'age', 'year']).to_csv(file_path, index=False)

In [26]:
df.head()

,measure,location,sex,age,cause,metric,year,mmr_rate
6464,25,548,2,8,376,3,1990,0.404898
6465,25,548,2,9,376,3,1990,0.474038
6466,25,548,2,10,376,3,1990,0.536445
6467,25,548,2,11,376,3,1990,0.928353
6468,25,548,2,12,376,3,1990,1.957516


In [32]:
all_measures

[['deaths_number', 'Deaths Number'],
 ['deaths_percent', 'Deaths Percent'],
 ['deaths_rate', 'Deaths Rate'],
 ['dalys_number', 'DALYs Number'],
 ['dalys_percent', 'DALYs Percent'],
 ['dalys_rate', 'DALYs Rate'],
 ['ylds_number', 'YLDs Number'],
 ['ylds_percent', 'YLDs Percent'],
 ['ylds_rate', 'YLDs Rate'],
 ['ylls_number', 'YLLs Number'],
 ['ylls_percent', 'YLLs Percent'],
 ['ylls_rate', 'YLLs Rate'],
 ['prevalence_number', 'Prevalence Number'],
 ['prevalence_percent', 'Prevalence Percent'],
 ['prevalence_rate', 'Prevalence Rate'],
 ['incidence_number', 'Incidence Number'],
 ['incidence_percent', 'Incidence Percent'],
 ['incidence_rate', 'Incidence Rate'],
 ['mmr_rate', 'MMR Rate']]

In [25]:
md['cause'].shape

(335, 5)

In [26]:
md['location'].shape

(436, 6)

In [27]:
md['sex'].shape

(3, 3)

In [28]:
md['age'].shape

(46, 6)

In [33]:
# entities

In [35]:
sexs = md['sex'].copy()

In [36]:
sexs

,sex_id,name,label
1,1,Male,Males
2,2,Female,Females
3,3,Both,Both sexes


In [40]:
sexs.columns = ['sex', 'name', 'short_name']
sexs[['sex', 'name']].to_csv('../../ddf--entities--sex.csv', index=False)

In [38]:
causes = md['cause'].copy()

In [42]:
causes.head()

,cause,label,name,medium_name,short_name
294,294,Total,All causes,All causes,All causes
295,295,A,"Communicable, maternal, neonatal, and nutritio...",Group I,Group I
296,296,A.1,HIV/AIDS and tuberculosis,HIV/AIDS & tuberculosis,HIV+TB
297,297,A.1.1,Tuberculosis,Tuberculosis,TB
298,298,A.1.2,HIV/AIDS,HIV/AIDS,HIV


In [41]:
causes.columns = ['cause', 'label', 'name', 'medium_name', 'short_name']
causes.to_csv('../../ddf--entities--cause.csv', index=False)

In [43]:
locations = md['location'].copy()

In [44]:
locations

,location_id,type,name,medium_name,short_name,enabled
1,1,NaN,Global,Global,Earth,NaN
10,10,NaN,Cambodia,Cambodia,Cambodia,NaN
100,100,NaN,High-income North America,High-income North America,HI N Am,NaN
101,101,NaN,Canada,Canada,Canada,NaN
102,102,NaN,United States,United States,USA,NaN
103,103,NaN,Latin America and Caribbean,Latin America and Caribbean,Latin Am & Caribbean,NaN
104,104,NaN,Caribbean,Caribbean,Caribbean,NaN
105,105,NaN,Antigua and Barbuda,Antigua and Barbuda,Antigua,NaN
106,106,NaN,The Bahamas,The Bahamas,Bahamas,NaN
107,107,NaN,Barbados,Barbados,Barbados,NaN


In [45]:
# don't include the custom region

In [46]:
locations = locations.iloc[:-1]

In [47]:
locations.shape

(435, 6)

In [48]:
locations = locations[['location_id', 'name', 'short_name', 'medium_name']]
locations.columns = ['location', 'name', 'short_name', 'medium_name']

In [49]:
locations.head()

,location,name,short_name,medium_name
1,1,Global,Earth,Global
10,10,Cambodia,Cambodia,Cambodia
100,100,High-income North America,HI N Am,High-income North America
101,101,Canada,Canada,Canada
102,102,United States,USA,United States


In [50]:
locations.to_csv('../../ddf--entities--location.csv', index=False)

In [63]:
ages = md['age'].copy()

In [65]:
ages.head()

,age_id,name,short_name,sort,plot,type
1,1,Under 5,<5,22,0,aggregate
10,10,25 to 29,25,10,1,specific
11,11,30 to 34,30,11,1,specific
12,12,35 to 39,35,12,1,specific
13,13,40 to 44,40,13,1,specific


In [66]:
ages = ages.sort_values(by='sort')[['age_id', 'name', 'short_name', 'type']]
ages.columns = ['age', 'name', 'short_name', 'type']

In [68]:
ages.to_csv('../../ddf--entities--age.csv', index=False)

In [55]:
# concepts

In [56]:
cont_cdf = pd.DataFrame(all_measures, columns=['concept', 'name'])

In [58]:
cont_cdf['concept_type'] = 'measure'

In [60]:
cont_cdf.to_csv('../../ddf--concepts--continuous.csv', index=False)

In [60]:
dis_cdf = pd.DataFrame([
    ['name', 'Name', 'string'],
    ['short_name', 'Short Name', 'string'],
    ['medium_name', 'Medium Name', 'string'],
    ['long_name', 'Long Name', 'string'],
    ['location', 'Location', 'entity_domain'],
    ['sex', 'Sex', 'entity_domain'],
    ['age', 'Age', 'entity_domain'],
    ['cause', 'Cause', 'entity_domain'],
    ['rei', 'Risk/Etiology/Impairment', 'entity_domain'],
    ['label', 'Label', 'string'],
    ['year', 'Year', 'time'],
    ['type', 'Type', 'string']
], columns=['concept', 'name', 'concept_type'])

In [61]:
dis_cdf.sort_values(by='concept').to_csv('../../ddf--concepts--discrete.csv', index=False)

In [11]:
# Risk dimension

In [12]:
rei = md['rei']

In [22]:
rei[rei['rei_id'] == 184]

,rei_id,type,name,short_name,enabled
184,184,etiology,Norovirus,Norovirus,1


In [19]:
causes = md['cause']

In [21]:
causes[causes['cause_id'] == 295]

,cause_id,cause,name,medium_name,short_name
295,295,A,"Communicable, maternal, neonatal, and nutritio...",Group I,Group I


In [23]:
# risk

In [27]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [8]:
rsd = '../source/risk/impairment/'

In [9]:
all_data = {}
all_measures = []
for n in os.listdir(rsd):
    print(n)
    if not n.endswith('.zip'):
        continue
    f = osp.join(rsd, n)
    zf = ZipFile(f)
    fname = osp.splitext(n)[0] + '.csv'
    data = pd.read_csv(zf.open(fname))
    data = data.drop(['upper', 'lower'], axis=1)
    # data = dd.from_pandas(data, npartitions=1)
    groups = data.groupby(by=['measure', 'metric'])
    for g in groups.groups:
        name  = measure[g[0]] + ' ' + metric[g[1]]
        concept = to_concept_id(name)
        all_measures.append([concept, name])
        df = groups.get_group(g)
        df = df.rename(columns={'val': concept})
        cols = ['location', 'sex', 'age', 'cause', 'rei', 'year', concept]
        
        # df = dd.from_pandas(df[cols], npartitions=10)
        df = df[cols]
        if concept in all_data.keys():
            all_data[concept] = all_data[concept].append(df, ignore_index=True)
        else:
            all_data[concept] = df
    zf.close()
    # break

IHME-GBD_2016_DATA-c6af0745-1.zip
IHME-GBD_2016_DATA-c6af0745-10.zip
IHME-GBD_2016_DATA-c6af0745-100.zip
IHME-GBD_2016_DATA-c6af0745-101.zip
IHME-GBD_2016_DATA-c6af0745-102.zip
IHME-GBD_2016_DATA-c6af0745-103.zip
IHME-GBD_2016_DATA-c6af0745-104.zip
IHME-GBD_2016_DATA-c6af0745-105.zip
IHME-GBD_2016_DATA-c6af0745-106.zip
IHME-GBD_2016_DATA-c6af0745-107.zip
IHME-GBD_2016_DATA-c6af0745-108.zip
IHME-GBD_2016_DATA-c6af0745-109.zip
IHME-GBD_2016_DATA-c6af0745-11.zip
IHME-GBD_2016_DATA-c6af0745-110.zip
IHME-GBD_2016_DATA-c6af0745-111.zip
IHME-GBD_2016_DATA-c6af0745-112.zip
IHME-GBD_2016_DATA-c6af0745-113.zip
IHME-GBD_2016_DATA-c6af0745-114.zip
IHME-GBD_2016_DATA-c6af0745-115.zip
IHME-GBD_2016_DATA-c6af0745-116.zip
IHME-GBD_2016_DATA-c6af0745-117.zip
IHME-GBD_2016_DATA-c6af0745-118.zip
IHME-GBD_2016_DATA-c6af0745-119.zip
IHME-GBD_2016_DATA-c6af0745-12.zip
IHME-GBD_2016_DATA-c6af0745-120.zip
IHME-GBD_2016_DATA-c6af0745-121.zip
IHME-GBD_2016_DATA-c6af0745-122.zip
IHME-GBD_2016_DATA-c6af0745-123.z

In [64]:
all_data.keys()

dict_keys(['dalys_number', 'dalys_percent', 'dalys_rate', 'ylds_number', 'ylds_percent', 'ylds_rate', 'ylls_number', 'ylls_percent', 'ylls_rate', 'deaths_number', 'deaths_percent', 'deaths_rate'])

In [12]:
all_data['deaths_percent'].shape

(5999986, 7)

In [65]:
all_data['dalys_number'].sex.unique()

array([1, 2, 3])

In [66]:
all_data['ylls_number'].age.unique()

array([22])

In [67]:
all_data['ylls_number'].year.unique()   ## I have checked in the gbd website, the data indeed is only available for these years.

array([1995, 1990, 2000, 2006, 2016, 2005, 2010])

In [68]:
all_data['ylls_number'].cause.unique()

array([294, 295, 301, 302, 322])

In [69]:
all_data['ylls_rate'].rei.unique()

array([173, 174, 175, 176, 177, 181, 182, 183, 184, 185, 187, 188, 189,
       190, 178, 179, 180])

In [16]:
1000000 * 100 / 12

8333333.333333333

In [18]:
(9162245 - 8333333) / 1000000

0.828912

In [40]:
# df = all_data.pop('deaths_number')
df = all_data['ylds_number']

In [20]:
# df.to_pickle('deaths_number.pkl')

In [23]:
for k in list(all_data.keys()):
    df = all_data.pop(k)
    df.to_pickle('{}.pkl'.format(k))

In [24]:
!open .

In [31]:
# df = pd.read_pickle('./ylds_percent.pkl')

In [41]:
df.columns

Index(['location', 'sex', 'age', 'cause', 'rei', 'year', 'ylds_number'], dtype='object')

In [10]:
!mkdir -p ../../../ddf--ihme--risk/impairment

In [14]:
for k, df in all_data.items():
    g = df.groupby(by='rei')
    for r in g.groups:
        df_ = g.get_group(r)
        df_ = df_[df_.sex == 3]
        by = ['location', 'sex', 'age', 'cause', f'rei-{r}', 'year']
        filename = f'ddf--datapoints--{k}--by--' + '--'.join(by) + '.csv'
        path = osp.join('../../../ddf--ihme--risk/impairment', filename)
        df_.to_csv(path, index=False)

In [15]:
!du -sh ../../../ddf--ihme--risk/impairment

1.6G	../../../ddf--ihme--risk/impairment


In [20]:
rei = md['rei']

In [22]:
rei[rei.name == 'Cholera']

,rei_id,type,name,short_name,enabled
173,173,etiology,Cholera,Cholera,1


In [45]:
rei.type.unique()

array(['risk', 'etiology', 'impairment', 'injury n-codes'], dtype=object)

In [62]:
rei_ent = rei[rei['type'] != 'injury n-codes']

In [63]:
rei_ent.head()

,rei_id,type,name,short_name,enabled
100,100,risk,Secondhand smoke,Secondhand smoke,1
101,101,risk,Alcohol and drug use,Alcohol & drug use,0
102,102,risk,Alcohol use,Alcohol use,1
103,103,risk,Drug use,Drug use,1
104,104,risk,Metabolic risks,Metabolic,0


In [65]:
rei_ent = rei_ent[['rei_id', 'type', 'name', 'short_name']]
rei_ent.columns = ['rei', 'type', 'name', 'short_name']

In [66]:
rei_ent.to_csv('../../ddf--entities--rei.csv', index=False)